In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import gspread
import time
import cv2
import numpy as np
from PIL import Image
import requests

from oauth2client.service_account import ServiceAccountCredentials 
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('suraping-381608-4562bacb42ee.json', scope)

def download_img(image, filename):
    r = requests.get(url=image, cookies=cookie, headers=headers, stream=True)
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(r.content)

date = 0 #当日は0、前日は1・・・
date2 = "08/19"
pmc = "06018001" # 店舗ID
SPREADSHEET_KEY = '1Sr1dL1MNr463Asw1Dk_Ap4S0mwnFM5KIroJvG6VsQ4w' # スプレッドシートのID
pstart = 1 # パチンコの1台目
sstart = 649 # スロットの1台目
wait = 0.01

user_id = "tomosvtg.1988+789@gmail.com"
user_pw = "paruparu23"

Template_p_path = "site777_2/template_p.png"
Template_s_path = "site777_2/template_s.png"
dot_point_p = 130000 / 432
dot_point_px = 60000 / 234
dot_point_s = 20000 / 432

mobile_emulation = { "deviceName": "iPhone 12 Pro" }

headers = {
    "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 16_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/108.0.5359.112 Mobile/15E148 Safari/604.1",
    "Referer" : "https://m.site777.jp/f/D3000.do?pmc=22021007&gc=2&dn=5019&urt=2173&mdc=120104&dsgk=0&dtdd=2"
}

options = Options()
options.add_experimental_option("mobileEmulation", mobile_emulation)
#options.add_argument('--headless')
options.binary_location = 'C:\\Program Files\\Google\\Chrome\\Application\\Chrome.exe'
chrome_service = ChromeService(executable_path='C:\\Users\\garden026\\WD\\114.0.5735.90\\chromedriver.exe')

gc = gspread.authorize(credentials)
wb = gc.open_by_key(SPREADSHEET_KEY)

worksheet = wb.worksheet("Config")
p4row = worksheet.cell(11,  23).value
p1row = worksheet.cell(11,  24).value
s20row = worksheet.cell(11,  28).value
s5row = worksheet.cell(11,  29).value

browser = webdriver.Chrome(service=chrome_service,options=options)
browser.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
params = {'cmd': 'Page.setDownloadBehavior', 'params': {'behavior': 'allow', 'downloadPath': 'site777'}}
command_result = browser.execute("send_command", params)

browser.get('https://m.site777.jp/f/A0100.do')
time.sleep(1)

browser.find_element("xpath",'/html/body/section[1]/div/div/p[3]/a').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/section/section[2]/div/div/p').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[1]/input').send_keys(user_id)
browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[2]/input').send_keys(user_pw)
time.sleep(1)

browser.execute_script('document.fmLogin.submit()')
time.sleep(1)

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(pstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

pmdc = browser.current_url.split("mdc=")[1]

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(sstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

smdc = browser.current_url.split("mdc=")[1]

ck = browser.get_cookies()
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "PO_SP_ID":
        po_sp_id = ck[k]["value"]
        k = 30
    k = k + 1
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "JSESSIONID":
        jsessionid = ck[k]["value"]
        k = 30
    k = k + 1

cookie = {"PO_SP_ID" : po_sp_id,
         "JSESSIONID" : jsessionid}

browser.close()

p4table = []
p1table = []
s20table = []
s5table = []

# Pachinko

dn = pstart
mdc = pmdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D2600S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 100:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<hatariSu>")[m + cc].split("</hatariSu>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_p_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 341 or y > 343:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 342 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        game = round(dot_point_px * ( xx - 10) , 0)
                        samai = round(dot_point_p * point ,0)-300
                        if xx==10:
                            samai = 0
                            game = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass
                    
                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                print(li)
                if unit == "400":
                    p4table.append(li)
                if unit == "125":
                    p1table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                    print(li)
                    if unit == "400":
                        p4table.append(li)
                    if unit == "125":
                        p1table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

# Slot

dn = sstart
mdc = smdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D3000S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 1000:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                game = jsonData.split("<totalGameCount>")[m + cc].split("</totalGameCount>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<regularCount>")[m + cc].split("</regularCount>")[0]
                art = jsonData.split("<artCount>")[m + cc].split("</artCount>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_s_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 332 or y > 334:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 333 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        samai = round(dot_point_s * point ,0)
                        if xx==10:
                            samai = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass

                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                print(li)
                if unit == "2173":
                    s20table.append(li)
                if unit == "561":
                    s5table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                    print(li)
                    if unit == "2173":
                        s20table.append(li)
                    if unit == "561":
                        s5table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

wb.values_append("'P4RAW'!F" + str(int(p4row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p4table})
wb.values_append("'P1RAW'!F" + str(int(p1row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p1table})
wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})
wb.values_append("'S5RAW'!F" + str(int(s5row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s5table})

['08/19', '025605', '1', '31282.0', '-3309.0', '27', '10']
['08/19', '025605', '2', '513.0', '1205.0', '3', '1']
['08/19', '025605', '3', '4359.0', '1506.0', '5', '4']
['08/19', '025605', '4', '8205.0', '-2106.0', '5', '2']
['08/19', '025605', '5', '21282.0', '904.0', '20', '6']
['08/19', '025605', '6', '14359.0', '2709.0', '18', '5']
['08/19', '025605', '7', '5385.0', '5418.0', '13', '2']
['08/19', '025605', '8', '10513.0', '-1203.0', '9', '5']
['08/19', '025605', '9', '17692.0', '1506.0', '21', '7']
['08/19', '025765', '10', '36923.0', '1205.0', '44', '13']
['08/19', '025765', '11', '35897.0', '-7823.0', '32', '11']
['08/19', '025765', '12', '28205.0', '-2406.0', '28', '14']
['08/19', '025765', '13', '35641.0', '6922.0', '52', '19']
['08/19', '025765', '14', '50000.0', '-4212.0', '49', '25']
['08/19', '025765', '15', '27692.0', '-1203.0', '32', '12']
['08/19', '025765', '16', '27436.0', '603.0', '32', '14']
['08/19', '025765', '17', '38974.0', '5719.0', '54', '21']
['08/19', '025765'

['08/19', '025553', '143', '13846.0', '-902.0', '9', '3']
['08/19', '025553', '144', '21282.0', '5117.0', '21', '8']
['08/19', '025700', '145', '3333.0', '-2707.0', '0', '0']
['08/19', '025469', '146', '12308.0', '6019.0', '13', '2']
['08/19', '025522', '147', '3333.0', '-2106.0', '1', '1']
['08/19', '025656', '148', '5641.0', '12038.0', '14', '3']
['08/19', '025707', '149', '12051.0', '603.0', '11', '5']
['08/19', '025803', '150', '5897.0', '-1203.0', '3', '3']
['08/19', '025764', '151', '16410.0', '16251.0', '19', '2']
['08/19', '025801', '152', '11026.0', '603.0', '9', '5']
['08/19', '025767', '153', '13846.0', '13543.0', '33', '2']
['08/19', '025483', '154', '15385.0', '-2707.0', '9', '1']
['08/19', '025625', '155', '14103.0', '-9629.0', '4', '1']
['08/19', '025701', '156', '6410.0', '-4513.0', '1', '1']
['08/19', '025794', '157', '9231.0', '27084.0', '23', '3']
['08/19', '025707', '158', '11026.0', '16251.0', '20', '4']
['08/19', '025745', '159', '11795.0', '2408.0', '40', '29']
[

['08/19', '025466', '287', '0', '0', '0', '0']
['08/19', '025466', '288', '0', '0', '0', '0']
['08/19', '025317', '289', '769.0', '-1203.0', '0', '0']
['08/19', '025317', '290', '769.0', '-1203.0', '0', '0']
['08/19', '025317', '291', '0', '0', '0', '0']
['08/19', '025605', '292', '6667.0', '-5115.0', '0', '0']
['08/19', '025605', '293', '4103.0', '-2406.0', '1', '1']
['08/19', '025605', '294', '11026.0', '3010.0', '14', '6']
['08/19', '025605', '295', '9231.0', '904.0', '11', '4']
['08/19', '025636', '296', '1795.0', '-1504.0', '0', '0']
['08/19', '025636', '297', '1795.0', '-1504.0', '0', '0']
['08/19', '025636', '298', '2821.0', '-2106.0', '0', '0']
['08/19', '025737', '299', '4615.0', '-3610.0', '0', '0']
['08/19', '025737', '300', '4359.0', '-3309.0', '0', '0']
['08/19', '025737', '301', '7949.0', '-2707.0', '3', '2']
['08/19', '025579', '302', '3333.0', '-902.0', '1', '1']
['08/19', '025579', '303', '0', '0', '0', '0']
['08/19', '025579', '304', '1795.0', '603.0', '1', '1']
['08/

['08/19', '025687', '439', '8718.0', '-7221.0', '0', '0']
['08/19', '025687', '440', '1538.0', '-1504.0', '0', '0']
['08/19', '025687', '441', '16923.0', '-14143.0', '0', '0']
['08/19', '025687', '442', '5641.0', '-2406.0', '2', '2']
['08/19', '025687', '443', '1282.0', '-1203.0', '0', '0']
['08/19', '025687', '444', '1026.0', '-1203.0', '0', '0']
['08/19', '025687', '445', '8205.0', '-6920.0', '0', '0']
['08/19', '025687', '446', '13333.0', '-10832.0', '0', '0']
['08/19', '025687', '447', '0', '0', '0', '0']
['08/19', '025687', '448', '7692.0', '3913.0', '8', '1']
['08/19', '025687', '449', '769.0', '-1203.0', '0', '0']
['08/19', '025687', '450', '14103.0', '-9027.0', '2', '2']
['08/19', '025665', '451', '30256.0', '-16249.0', '9', '6']
['08/19', '025665', '452', '42564.0', '-15346.0', '16', '6']
['08/19', '025665', '453', '23077.0', '-1504.0', '12', '3']
['08/19', '025665', '454', '35641.0', '22871.0', '35', '7']
['08/19', '025665', '455', '12821.0', '4515.0', '10', '1']
['08/19', '0

['08/19', '025822', '586', '513.0', '-1203.0', '0', '0']
['08/19', '025313', '587', '2051.0', '-1504.0', '0', '0']
['08/19', '025304', '588', '513.0', '-1203.0', '0', '0']
['08/19', '025668', '589', '3333.0', '-902.0', '7', '1']
['08/19', '025538', '590', '0', '0', '0', '0']
['08/19', '025758', '591', '0', '0', '0', '0']
['08/19', '025554', '592', '5897.0', '-2106.0', '3', '2']
['08/19', '025306', '593', '1026.0', '-1203.0', '0', '0']
['08/19', '025806', '594', '7179.0', '-5416.0', '0', '0']
['08/19', '025606', '595', '7949.0', '13242.0', '16', '4']
['08/19', '025606', '596', '2051.0', '-1805.0', '0', '0']
['08/19', '025606', '597', '4615.0', '-2707.0', '1', '1']
['08/19', '025606', '598', '3846.0', '7524.0', '7', '2']
['08/19', '025606', '599', '2051.0', '-1805.0', '0', '0']
['08/19', '025606', '600', '11795.0', '-9629.0', '0', '0']
['08/19', '025606', '601', '3077.0', '-2707.0', '0', '0']
['08/19', '025606', '602', '7692.0', '603.0', '5', '2']
['08/19', '025606', '603', '11282.0', '-

['08/19', '120053', '727', '1156', '880.0', '0', '2', '21']
['08/19', '119940', '728', '3338', '370.0', '11', '0', '10']
['08/19', '119940', '729', '835', '-231.0', '1', '0', '2']
['08/19', '119940', '730', '3538', '1204.0', '10', '0', '12']
['08/19', '120094', '731', '7', '0', '0', '0', '--']
['08/19', '120094', '732', '10', '0', '0', '0', '--']
['08/19', '120094', '733', '8', '0', '0', '0', '--']
['08/19', '120080', '734', '4577', '3565.0', '31', '0', '74']
['08/19', '120080', '735', '73', '0', '0', '0', '--']
['08/19', '120080', '736', '1802', '-1019.0', '17', '0', '9']
['08/19', '120073', '737', '9271', '1944.0', '0', '34', '64']
['08/19', '120073', '738', '1553', '-787.0', '0', '4', '7']
['08/19', '120073', '739', '7951', '3472.0', '0', '41', '70']
['08/19', '120073', '740', '1826', '-1620.0', '0', '3', '5']
['08/19', '120073', '741', '8253', '-509.0', '0', '35', '54']
['08/19', '120073', '742', '3756', '-1528.0', '0', '13', '18']
['08/19', '120073', '743', '7729', '3889.0', '0', 

['08/19', '120010', '863', '4503', '463.0', '17', '17', '--']
['08/19', '120010', '864', '5131', '1759.0', '25', '16', '--']
['08/19', '120010', '865', '8326', '1435.0', '30', '37', '--']
['08/19', '120010', '866', '2613', '185.0', '12', '4', '--']
['08/19', '120010', '867', '7217', '1389.0', '33', '23', '--']
['08/19', '119916', '868', '614', '-370.0', '1', '1', '--']
['08/19', '119916', '869', '5298', '-185.0', '20', '11', '--']
['08/19', '119916', '870', '3212', '1667.0', '19', '6', '--']
['08/19', '119916', '871', '559', '324.0', '4', '0', '--']
['08/19', '119916', '872', '1401', '324.0', '8', '2', '--']
['08/19', '120033', '873', '2772', '926.0', '11', '5', '--']
['08/19', '120033', '874', '2565', '-1250.0', '5', '4', '--']
['08/19', '120033', '875', '1473', '370.0', '5', '4', '--']
['08/19', '120033', '876', '2465', '278.0', '8', '6', '--']
['08/19', '120103', '877', '3149', '-2083.0', '20', '3', '8']
['08/19', '120103', '878', '2948', '5231.0', '50', '8', '11']
['08/19', '120103

['08/19', '119996', '997', '2162', '-972.0', '5', '5', '--']
['08/19', '119996', '998', '2338', '-648.0', '6', '7', '--']
['08/19', '119996', '999', '3355', '787.0', '17', '6', '--']
['08/19', '119996', '1000', '2252', '509.0', '12', '4', '--']
['08/19', '119996', '1001', '328', '-139.0', '1', '0', '--']
['08/19', '119996', '1002', '7913', '1759.0', '34', '34', '--']
['08/19', '119996', '1003', '549', '93.0', '2', '2', '--']
['08/19', '119996', '1004', '1360', '139.0', '6', '2', '--']
['08/19', '119996', '1005', '2035', '-417.0', '6', '5', '--']


{'spreadsheetId': '1Sr1dL1MNr463Asw1Dk_Ap4S0mwnFM5KIroJvG6VsQ4w',
 'updates': {'spreadsheetId': '1Sr1dL1MNr463Asw1Dk_Ap4S0mwnFM5KIroJvG6VsQ4w',
  'updatedRange': 'S5RAW!F5978:M6013',
  'updatedRows': 36,
  'updatedColumns': 8,
  'updatedCells': 288}}